### Install and import library

In [1]:
pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 798.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 88.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 9.8 MB/s eta 0:00:00


In [2]:
import getpass
import boto3
from getpass import getpass

### Use created IAM user to have access key to connect the AWS

In [3]:
# Create a IAM User ImageLinkLoader

# Securely input the Account ID and Role Name
aws_access_key_id = getpass("Enter your IAM Access Key ID: ")
aws_secret_access_key = getpass("Enter your IAM Secret Access Key ID: ")

# Create an S3 client with the IAM user's credentials
s3_client = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)

Enter your IAM Access Key ID: ··········
Enter your IAM Secret Access Key ID: ··········


In [28]:
region_name = 'us-east-2'  # e.g., 'us-east-1'
bucket_name = 'taotravelbucket'
folder_path = 'orig/us/alaska/fairbanks/'  # e.g., 'images/' with /

### Load Images links and all other info into tuple

In [29]:
def list_images(bucket, folder):
    response = s3_client.list_objects_v2(Bucket=bucket, Prefix=folder)
    image_details = []
    total_objects = 0  # Counter for all objects in the folder
    checked_objects = 0  # Counter for objects with valid extensions
    image_type = ['jpg', 'jpeg', 'png', 'gif', 'bmp', 'tiff', 'webp', 'heic']

    if 'Contents' in response:
        for item in response['Contents']:
            total_objects += 1
            file_name = item['Key']
            ext = file_name.split('.')[-1].lower()
            if ext in image_type:
                checked_objects += 1
                # Split the file path to get the location details
                path_parts = file_name.split('/')
                # Ensure the path has enough parts to prevent index errors
                if len(path_parts) >= 4:
                    country = path_parts[1]
                    province = path_parts[2]
                    city = path_parts[3]
                    sub_city = ''
                    orig_name = path_parts[-1]
                    if len(path_parts) > 5:
                        sub_city = path_parts[4]  # Adjust as per your path structure

                    # Construct the original image URL
                    orig_image_url = f'https://{bucket}.s3.{region_name}.amazonaws.com/{file_name}'

                    # Construct the modified image URL
                    comp_file_name = file_name.replace('orig', 'comp', 1).rsplit('.', 1)[0] + '.jpeg'
                    comp_image_url = f'https://{bucket}.s3.{region_name}.amazonaws.com/{comp_file_name}'

                    # Append the details tuple to the list
                    image_details.append((country, province, city, sub_city, orig_name, orig_image_url, comp_image_url))

            else:
              print('Error file: ', file_name)

    print(f"Total objects in folder: {total_objects}")
    print(f"Objects with valid extensions: {checked_objects}")

    return image_details

Anchorage, Palmer, Fairbanks

In [30]:
# Retrieve and print the list of image details
image_detail_pairs = list_images(bucket_name, folder_path)

#for details in image_detail_pairs:
#    print(details)
print(image_detail_pairs[0])

Error file:  orig/us/alaska/fairbanks/.DS_Store
Total objects in folder: 362
Objects with valid extensions: 361
('us', 'alaska', 'fairbanks', '', 'DSC00001.JPG', 'https://taotravelbucket.s3.us-east-2.amazonaws.com/orig/us/alaska/fairbanks/DSC00001.JPG', 'https://taotravelbucket.s3.us-east-2.amazonaws.com/comp/us/alaska/fairbanks/DSC00001.jpeg')


### Convert to SQL Query

In [31]:
# Insert Query in SQL
query = "INSERT INTO travel_image (country, province, city, sub_city, orig_name, orig_link, comp_link) VALUES \n"

# Formatting the values for SQL insertion
values_list = []
for details in image_detail_pairs:
    country, province, city, sub_city, orig_name, orig_link, comp_link = [f"'{detail}'" for detail in details]
    values_str = f"({country}, {province}, {city}, {sub_city}, {orig_name}, {orig_link}, {comp_link})"
    values_list.append(values_str)

# Join the values and complete the query
query += ",\n".join(values_list) + ";"

In [37]:
print(query[0:337])

INSERT INTO travel_image (country, province, city, sub_city, orig_name, orig_link, comp_link) VALUES 
('us', 'alaska', 'fairbanks', '', 'DSC00001.JPG', 'https://taotravelbucket.s3.us-east-2.amazonaws.com/orig/us/alaska/fairbanks/DSC00001.JPG', 'https://taotravelbucket.s3.us-east-2.amazonaws.com/comp/us/alaska/fairbanks/DSC00001.jpeg'),
